## Decision Tree Regressor in Python - sklearn
- A decision tree is a flowchart-like diagram that shows the various outcomes from a series of decisions. It can be used as a decision-making tool, for research analysis, or for planning strategy
- Dataset was obtained in [Kaggle](https://www.kaggle.com/datasets/hellbuoy/car-price-prediction).  
- Please check the conclusion in the end.

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split as tts
from sklearn import metrics 
import random
np.random.seed(1)

get_ipython().run_line_magic('matplotlib', 'inline')

import warnings
warnings.filterwarnings('ignore')

## 1. EDA, Data Manipulation and Feature Engineering

In [2]:
df = pd.read_csv('Bases de Dados\carprice.csv', index_col = 'car_ID')
print(df.shape)
df.head()

(205, 25)


,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
car_ID,,,,,,,,,,,,,,,,,,,,,
1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [3]:
df.drop(['symboling', 'CarName'], axis=1, inplace = True)
df.head()

,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
car_ID,,,,,,,,,,,,,,,,,,,,,
1,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
2,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
3,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
4,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
5,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [4]:
for i in df.columns:
    datatype = df[i].unique()
    print('    ',i,': ', datatype)

     fueltype :  ['gas' 'diesel']
     aspiration :  ['std' 'turbo']
     doornumber :  ['two' 'four']
     carbody :  ['convertible' 'hatchback' 'sedan' 'wagon' 'hardtop']
     drivewheel :  ['rwd' 'fwd' '4wd']
     enginelocation :  ['front' 'rear']
     wheelbase :  [ 88.6  94.5  99.8  99.4 105.8  99.5 101.2 103.5 110.   88.4  93.7 103.3
  95.9  86.6  96.5  94.3  96.  113.  102.   93.1  95.3  98.8 104.9 106.7
 115.6  96.6 120.9 112.  102.7  93.   96.3  95.1  97.2 100.4  91.3  99.2
 107.9 114.2 108.   89.5  98.4  96.1  99.1  93.3  97.   96.9  95.7 102.4
 102.9 104.5  97.3 104.3 109.1]
     carlength :  [168.8 171.2 176.6 177.3 192.7 178.2 176.8 189.  193.8 197.  141.1 155.9
 158.8 157.3 174.6 173.2 144.6 150.  163.4 157.1 167.5 175.4 169.1 170.7
 172.6 199.6 191.7 159.1 166.8 169.  177.8 175.  190.9 187.5 202.6 180.3
 208.1 199.2 178.4 173.  172.4 165.3 170.2 165.6 162.4 173.4 181.7 184.6
 178.5 186.7 198.9 167.3 168.9 175.7 181.5 186.6 156.9 157.9 172.  173.5
 173.6 158.7 169.7 166.

#### Transforming some of the categorical columns to Numeric/Boolean

In [5]:
print(df['fueltype'].value_counts(), '\n')
for i in range(0, len(df['fueltype'])):
    if df['fueltype'].iloc[i] == 'gas':
        df['fueltype'].iloc[i] = 1
    else:
        df['fueltype'].iloc[i] = 0
df['fueltype'].value_counts()

gas       185
diesel     20
Name: fueltype, dtype: int64 



1    185
0     20
Name: fueltype, dtype: int64

In [6]:
print(df['aspiration'].value_counts(), '\n')
for i in range(0, len(df['aspiration'])):
    if df['aspiration'].iloc[i] == 'turbo':
        df['aspiration'].iloc[i] = 1
    else:
        df['aspiration'].iloc[i] = 0
df['aspiration'].value_counts()

std      168
turbo     37
Name: aspiration, dtype: int64 



0    168
1     37
Name: aspiration, dtype: int64

In [7]:
print(df['doornumber'].value_counts(), '\n')
for i in range(0, len(df['doornumber'])):
    if df['doornumber'].iloc[i] == 'two':
        df['doornumber'].iloc[i] = 1
    else:
        df['doornumber'].iloc[i] = 0
df['doornumber'].value_counts()

four    115
two      90
Name: doornumber, dtype: int64 



0    115
1     90
Name: doornumber, dtype: int64

In [8]:
print(df['enginelocation'].value_counts(), '\n')
for i in range(0, len(df['enginelocation'])):
    if df['enginelocation'].iloc[i] == 'rear':
        df['enginelocation'].iloc[i] = 1
    else:
        df['enginelocation'].iloc[i] = 0
df['enginelocation'].value_counts()   

front    202
rear       3
Name: enginelocation, dtype: int64 



0    202
1      3
Name: enginelocation, dtype: int64

In [9]:
print(df['cylindernumber'].value_counts(), '\n')
for i in range(0, len(df['cylindernumber'])):
    if df['cylindernumber'].iloc[i] == 'four':
        df['cylindernumber'].iloc[i] = 4
    elif df['cylindernumber'].iloc[i] == 'six':
        df['cylindernumber'].iloc[i] = 6
    elif df['cylindernumber'].iloc[i] == 'five':
        df['cylindernumber'].iloc[i] = 5
    elif df['cylindernumber'].iloc[i] == 'eight':
        df['cylindernumber'].iloc[i] = 8
    elif df['cylindernumber'].iloc[i] == 'two':
        df['cylindernumber'].iloc[i] = 2
    elif df['cylindernumber'].iloc[i] == 'three':
        df['cylindernumber'].iloc[i] = 3
    elif df['cylindernumber'].iloc[i] == 'twelve':
        df['cylindernumber'].iloc[i] = 12
df['cylindernumber'].value_counts()   

four      159
six        24
five       11
eight       5
two         4
three       1
twelve      1
Name: cylindernumber, dtype: int64 



4     159
6      24
5      11
8       5
2       4
3       1
12      1
Name: cylindernumber, dtype: int64

#### Dropping remaining categorical columns

In [10]:
df = df.loc[:, df.dtypes != 'object']
df

,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
car_ID,,,,,,,,,,,,,,
1,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,111,5000,21,27,13495.0
2,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,111,5000,21,27,16500.0
3,94.5,171.2,65.5,52.4,2823,152,2.68,3.47,9.0,154,5000,19,26,16500.0
4,99.8,176.6,66.2,54.3,2337,109,3.19,3.40,10.0,102,5500,24,30,13950.0
5,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,8.0,115,5500,18,22,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,109.1,188.8,68.9,55.5,2952,141,3.78,3.15,9.5,114,5400,23,28,16845.0
202,109.1,188.8,68.8,55.5,3049,141,3.78,3.15,8.7,160,5300,19,25,19045.0
203,109.1,188.8,68.9,55.5,3012,173,3.58,2.87,8.8,134,5500,18,23,21485.0


In [11]:
df.describe()

,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,3.329756,3.255415,10.142537,104.117073,5125.121951,25.219512,30.751220,13276.710571
std,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,0.270844,0.313597,3.972040,39.544167,476.985643,6.542142,6.886443,7988.852332
min,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,13.000000,16.000000,5118.000000
25%,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,3.150000,3.110000,8.600000,70.000000,4800.000000,19.000000,25.000000,7788.000000
50%,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,3.310000,3.290000,9.000000,95.000000,5200.000000,24.000000,30.000000,10295.000000
75%,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,3.580000,3.410000,9.400000,116.000000,5500.000000,30.000000,34.000000,16503.000000
max,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,3.940000,4.170000,23.000000,288.000000,6600.000000,49.000000,54.000000,45400.000000


## 2. Regression Decision Tree - sklearn

#### Splitting data in train and test and setting target.

In [12]:
x = df.drop('price', axis = 1)
y = df['price']

x_train, x_test, y_train, y_test = tts(x,y, random_state=0, test_size=0.3)

In [13]:
from sklearn.tree import DecisionTreeRegressor

model_dtr = DecisionTreeRegressor()

model_dtr = model_dtr.fit(x_train, y_train)

In [14]:
preds_dtr = model_dtr.predict(x_test)

- How well the model learned ?  
    if the MAE value is 10% the average value of the target, the model learned well.

In [15]:
MAE = metrics.mean_absolute_error(y_test, preds_dtr)
print('mean_absolute_error: ', MAE)

test_avg = round(y_test.mean(), 2)
print("Average Price: ", test_avg)

MAE_ratio = round(MAE/test_avg * 100, 2)
print("MAE percentage in relation to the average Price: ", MAE_ratio)

mean_absolute_error:  1870.717741935484
Average Price:  13223.73
MAE percentage in relation to the average Price:  14.15


#### Predicting prices with Linear Regression Model

In [16]:
from sklearn.linear_model import LinearRegression

model_reglin = LinearRegression()

model_reglin = model_reglin.fit(x_train, y_train)

preds_reglin = model_reglin.predict(x_test)

# -------------------------------------------------------------

MAE_reglin = metrics.mean_absolute_error(y_test, preds_reglin)
print('mean_absolute_error: ', MAE_reglin)

test_avg_reglin = round(y_test.mean(), 2)
print("Average Price: ", test_avg_reglin)

MAE_ratio_reglin = round(MAE_reglin/test_avg_reglin * 100, 2)
print("MAE percentage in relation to the average Price: ", MAE_ratio_reglin)

mean_absolute_error:  2379.512739504234
Average Price:  13223.73
MAE percentage in relation to the average Price:  17.99
